## What Does It Take to Be An Expert At Python
Notebook based off James Powell's talk at PyData 2017'
https://www.youtube.com/watch?v=7lmCu8wz8ro


* Dunder methods are data-model methods
* whenever we want to implement a behavior, there's a pattern
* Top-level functions or top-level syntax
* There's a corrsponding data model function 
    
* we can implement the notion of size in this class, implement dunder len 
* whenever we want to implement a custom behavior on a python object:
* Implement a dunder method which ties to some top level syntax or some top level function 
   


In [7]:
# some behavior that I want to implement -> write some __ function __
# top-level function or top-level syntax -> corresponding __
# x + y -> __add__
# init x -> __init__
# repr(x) --> __repr__
# x() -> __call__

class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    
    def __add__(self, other):
        return Polynomial(*(x + y for x, y in zip(self.coeffs, other.coeffs)))
    
    def __len__(self):
        return len(self.coeffs)
    
    def __call__(self):
        pass

In [8]:
p1 = Polynomial(1,2,3)
p2 = Polynomial(3,4,5)

In [9]:
p1 + p2

Polynomial(*(4, 6, 8))

In [10]:
len(p1)

3

### 3 Core Patterns to understand object orientation
* Protocol view of python 
* Built-in inheritance protocol (where to go)
* Caveats around how object orientation in python works

### Metaclasses
* A feature that is warned against usually
* Very useful when grasped 


In [11]:
# File 1: library
class Base:
    def food(self):
        return 'foo'



In [12]:
# file 2: users
# from library import Base
assert hasattr(Base, 'foo'), "You broke it"
class Derived(Base):
        def bar(self):
            return self.foo()
    

AssertionError: You broke it

* This code could break if there's no foo method!
* if the person writing the library removes the foo method
* We can write a test that calls bar()
* Simpler solution? 
    * add an assert to check existence before the class is defined = invoking constrains on the library level


#### Now assume you're the person working on the Base class (file 1)
* How do you make sure that the user (file2), that the bar method is implemenet. Because if its not, the code will break
* We can't use the assert method in this case
* There are 3 common answers to this: 
    * metaclasses
    * Try Catch (this will catch it at runtime)
    * __Build_Class__

In [13]:
# File 1 - library.py

class Base:
    def foo(self):
        return self.bar()

In [14]:
# File2 - user.py

assert hasattr(Base, 'foo'), "you broke it, you fool!"

class Derived(Base):
    def bar(self):
        return 'bar'

In [15]:
# In python, a class is a runtime excutable code
# unlike C++, a class is : Here's a collection of bits
# You can define a class in a loop in python
for _ in range(10):
    class Base: pass # has no use 

# Or

class Base():
    for _ in range(10):
        def Bar(self):
            pass
# Python still accepts the above 

In [16]:
def _():
    class Base:
        pass

from dis import dis #disassemble

In [17]:
dis(_)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object Base at 0x0000026C52A19D20, file "<ipython-input-16-10c6f13882ab>", line 2>)
              4 LOAD_CONST               2 ('Base')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('Base')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (Base)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


In [18]:
# Catch Building of Classes

class Base:
    def foo(self):
        return self.bar()

old_bc = __build_class__
def my_bc(*a, **kw):
    print('my buildclass ->', a, kw)
    return old_bc(*a, **kw)
import builtins
builtins.__build_class__ = my_bc

In [19]:
# Catch Building of Classes

class Base:
    def foo(self):
        return self.bar()

    
old_bc = __build_class__
def my_bc(fun, name, base=None, **kw):
    if base is Base:
                print('Check if bar method defined')
    if base is not None:
        return old_bc(fun, name, base, **kw)
    return old_bc(fun, name, **kw)

import builtins
builtins.__build_class__ = my_bc

my buildclass -> (<function Base at 0x0000026C52A13F28>, 'Base') {}


In [20]:
import builtins
import importlib
importlib.reload(builtins)

<module 'builtins' (built-in)>

### The above is to demonstrate the functionality, not the actual way of solving it
* There are two features in Python that are used to solve the problem of enforcing constraints from dervied class to base class:

    * Metaclasses: classes that derive from type. Allow you to intercept the construction of derived types!
    * Enforcing constraints down the class hierarchy from a base class to a derived class

In [21]:
class BaseMeta(type):
    def __new__(cls, name, bases, body):
        print('BaseMeta.__new__', cls, name, bases, body)
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()

my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52977F28>, 'BaseMeta', <class 

RecursionError: maximum recursion depth exceeded

In [22]:
class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if not 'bar' in body:
            raise TypeError('bad user class')
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()

my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9EA0>, 'BaseMeta', <class 

RecursionError: maximum recursion depth exceeded

In [23]:

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if name != 'Base' and not 'bar' in body:
            raise TypeError('bad user class')
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()
    
    def __init_subclass__(*a, **kw):
        print('init_subclass', a, kw)
        return super().__init_subclass__(*a, **kw)

my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 

my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 'type'>) {}
my buildclass -> (<function BaseMeta at 0x0000026C52BC9F28>, 'BaseMeta', <class 

RecursionError: maximum recursion depth exceeded

In [24]:
help(Base.__init_subclass__)


Help on built-in function __init_subclass__:

__init_subclass__(...) method of builtins.type instance
    This method is called when a class is subclassed.
    
    The default implementation does nothing. It may be
    overridden to extend subclasses.



## Decorators

In [34]:
def add(x, y=10):
    return x + y

In [35]:
add(10, 20)


30

In [36]:
add

<function __main__.add(x, y=10)>

In [37]:
# Name of function
add.__name__

'add'

In [38]:
# What module function is assigned to
add.__module__

'__main__'

In [39]:
# Default values
add.__defaults__

(10,)

In [40]:
# Byte code for function
add.__code__.co_code

b'|\x00|\x01\x17\x00S\x00'

In [41]:
# Variable names function interacts with
add.__code__.co_varnames

('x', 'y')

In [42]:
# Source code
from inspect import getsource
getsource(add)

'def add(x, y=10):\n    return x + y\n'

In [43]:
print(getsource(add))


def add(x, y=10):
    return x + y



In [44]:
# What file are you in? 
from inspect import getfile
getfile(add)

'<ipython-input-34-6451af1f1873>'

In [45]:
from inspect import getmodule
getmodule(add)

<module '__main__'>

In [46]:
print('add(10)', add(10))
print('add(20, 30)', add(20, 30))
print('add("a", "b")', add("a", "b"))

add(10) 20
add(20, 30) 50
add("a", "b") ab


In [48]:
#Count how long it took to run
def add_timer(x, y=10):
    before = time()
    rv = x + y
    after = time()
    print('elapsed:', after - before)
    return rv

In [49]:
print('add(10)', add_timer(10))
print('add(20, 30)', add_timer(20, 30))
print('add("a", "b")', add_timer("a", "b"))

NameError: name 'time' is not defined